# Depreciation to FERC 1 Connection

This stage in the connection process is very much under constuction

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import pudl
import sqlalchemy as sa
import logging
import sys

from datetime import date

import pudl_rmi
from pudl_rmi.connect_deprish_to_ferc1 import *

import pudl_rmi.model_inputs

import warnings
warnings.filterwarnings('ignore')

In [3]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

pd.options.display.max_columns = None
pd.options.display.max_rows = 60

## Make the output via rmi_out

In [4]:
# pudl output object
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings["pudl_db"])
pudl_out = pudl.output.pudltabl.PudlTabl(
    pudl_engine,freq='AS',
    fill_fuel_cost=True,
    roll_fuel_cost=True,
    fill_net_gen=True,
)
rmi_out = pudl_rmi.coordinate.Output(
    pudl_out,
)

In [65]:
fe = rmi_out.grab_ferc1_to_eia(clobber=False)

Reading the FERC to EIA connection from /Users/christinagosnell/code/rmi-ferc1-eia/outputs/ferc1_eia.pkl.gz


In [482]:
ppl, d, de, fe, scaled_df = rmi_out.grab_all(clobber_all=False)

Reading the plant part list from /Users/christinagosnell/code/rmi-ferc1-eia/outputs/plant_parts_eia.pkl.gz
Grabbing depreciation study output from /Users/christinagosnell/code/rmi-ferc1-eia/outputs/deprish.pkl.gz
Reading the FERC to EIA connection from /Users/christinagosnell/code/rmi-ferc1-eia/outputs/ferc1_eia.pkl.gz
Reading the depreciation to FERC connection from /Users/christinagosnell/code/rmi-ferc1-eia/outputs/deprish_ferc1.pkl.gz


### Modernization of Duke
like grid mod.... but worse

In [580]:
def fake_duke_deprish_eia_for_mod(de):
    """Temp function to fake Duke's deprish records for modernization."""
    logger.info("Adding fake years of Duke data....")
    # omigosh bc there are double in here some how!
    if len(de.filter(like='record_id_eia_fuzzy').columns) == 2:
        de = de.drop(columns=['record_id_eia_fuzzy'])
    cols_to_keep = [
        'plant_part_name', 'utility_name_ferc1', 'report_year', 'report_date',
        'plant_name_match', 'record_id_eia', 'line_id', 'utility_id_pudl',
        'data_source'
    ]
    fake_year_dfs = []
    de_2018 = (
        de[
            de.utility_id_pudl.isin([90, 97])
            & (de.report_date.dt.year == 2018)
        ]
    )
    for fake_year in [2019, 2020]:
        de_fake_new_year = (
            de_2018.copy()
            .assign(
                report_year=fake_year,
                report_date=pd.to_datetime(fake_year, format="%Y")
            )
            .replace(
                {"record_id_eia": "_2018_",
                 "line_id": "2018_",},
                {"record_id_eia": f"_{fake_year}_",
                 "line_id": f"{fake_year}_"},
                regex=True
            )
            [cols_to_keep]
            .reset_index()
        )
        fake_year_dfs.append(de_fake_new_year)
    de_faked = pd.concat([de] + fake_year_dfs, ignore_index=True)
    assert (~de_faked[de_faked.report_date.dt.year == 2020].empty)
    return de_faked

de = fake_duke_deprish_eia_for_mod(de).pipe(pudl.helpers.convert_cols_dtypes, 'eia')
#fe = calc_annual_capital_addts_ferc1(fe)

Adding fake years of Duke data....


### Keep T&D

In [231]:
d_non_plant = d[~d.line_id.isin(de.line_id.unique())]
logger.info(
    f"Merging back in {len(d_non_plant)/len(d):.1%} of deprish records "
    f"AKA {d_non_plant.plant_balance.sum()/d.plant_balance.sum():.1%} of plant_balance "
    f"AKA {d_non_plant.unaccrued_balance.sum()/d.unaccrued_balance.sum():.1%} of unaccrued_balance!"
)

assert(not [c for c in d.columns if c not in de])

Merging back in 46.0% of deprish records AKA 55.6% of plant_balance AKA 75.0% of unaccrued_balance!


In [232]:
[c for c in de.columns if c not in d]

['report_year',
 'plant_name_match',
 'record_id_eia',
 'record_id_eia_fuzzy',
 'record_id_eia_fuzzy',
 'record_id_eia_override',
 'record_id_eia_override10',
 'record_id_eia_override11',
 'record_id_eia_override2',
 'record_id_eia_override3',
 'record_id_eia_override4',
 'record_id_eia_override5',
 'record_id_eia_override6',
 'record_id_eia_override7',
 'record_id_eia_override8',
 'record_id_eia_override9',
 'ownership',
 'fraction_owned',
 'ownership_dupe',
 'generator_id',
 'utility_id_ferc1',
 'ferc_acct_name',
 'utility_id_eia',
 'prime_mover_code',
 'energy_source_code_1',
 'unit_id_pudl',
 'plant_part',
 'plant_part_name_match',
 'record_count',
 'operational_status',
 'technology_description',
 'plant_name_eia',
 'plant_name_new',
 'true_gran_name_match',
 'operational_status_pudl',
 'matches']

In [ ]:
d.unaccrued_balance

In [259]:
fe[fe.utility_id_pudl.isin([97,90])].utility_id_pudl

119      90
120      90
121      90
122      90
123      90
         ..
51456    90
51473    90
51474    90
51490    90
51491    90
Name: utility_id_pudl, Length: 1906, dtype: Int64

In [220]:
de

,plant_part_name,utility_name_ferc1,report_year,plant_name_match,record_id_eia,record_id_eia_fuzzy,record_id_eia_fuzzy,record_id_eia_override,record_id_eia_override10,record_id_eia_override11,record_id_eia_override2,record_id_eia_override3,record_id_eia_override4,record_id_eia_override5,record_id_eia_override6,record_id_eia_override7,record_id_eia_override8,record_id_eia_override9,ownership,fraction_owned,ownership_dupe,generator_id,utility_id_ferc1,ferc_acct_name,report_date,utility_id_eia,prime_mover_code,energy_source_code_1,unit_id_pudl,plant_part,plant_id_eia,plant_part_name_match,record_count,operational_status,technology_description,utility_id_pudl,plant_name_eia,plant_name_new,true_gran_name_match,operational_status_pudl,book_reserve,book_reserve_w_common,common,data_source,depreciation_annual_epxns,depreciation_annual_epxns_w_common,depreciation_annual_rate,line_id,matches,net_removal_rate,net_salvage,net_salvage_rate,net_salvage_w_common,plant_balance,plant_balance_w_common,remaining_life_avg,remaining_life_avg_old,reserve_rate,service_life_avg,unaccrued_balance,unaccrued_balance_w_common
0,cayuga unit 1,"Duke Energy Indiana, LLC",2018.0,cayuga 1,1001_1_2018_plant_unit_total_15470,1001_1_2018_plant_unit_owned_15470,1001_1_2018_plant_unit_total_15470,1001_1_2018_plant_unit_total_15470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,total,1.0,False,1,144,Steam,2018-01-01,15470,ST,BIT,1,plant_unit,1001,plant_unit,2.0,existing,Conventional Steam Coal,92,Cayuga,Cayuga 1,True,operating,252273039.0,3.206063e+08,False,PUC,39728822.0,5.700070e+07,0.072917,2018_1001_cayuga unit 1_steam_92_PUC,"(cayuga 1, 76)",NaN,-39704796.19,-0.070000,-5.446970e+07,5.672114e+08,7.781386e+08,8.982381,8.974344,0.414600,NaN,354643134.0,5.120020e+08
1,cayuga inland container,"Duke Energy Indiana, LLC",2018.0,None,1001_2018_plant_total_15470,NaN,1001_2018_plant_total_15470,1001_2018_plant_total_15470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,total,1.0,False,<NA>,144,NaN,2018-01-01,15470,NaN,<NA>,<NA>,plant,1001,plant,7.0,existing,NaN,92,Cayuga,Cayuga,True,operating,2759781.0,NaN,True,PUC,123174.0,NaN,NaN,2018_1001_cayuga inland container_steam_92_PUC,"(cayuga other, 63)",NaN,-249966.80,NaN,NaN,3.570952e+06,NaN,NaN,NaN,NaN,NaN,1061138.0,NaN
2,cayuga unit 2,"Duke Energy Indiana, LLC",2018.0,cayuga 2,1001_2_2018_plant_unit_total_15470,1001_2_2018_plant_unit_owned_15470,1001_2_2018_plant_unit_total_15470,1001_2_2018_plant_unit_total_15470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,total,1.0,False,2,144,Steam,2018-01-01,15470,ST,BIT,2,plant_unit,1001,plant_unit,2.0,existing,Conventional Steam Coal,92,Cayuga,Cayuga 2,True,operating,237435190.0,2.758198e+08,False,PUC,34497367.0,4.567329e+07,0.068672,2018_1001_cayuga unit 2_steam_92_PUC,"(cayuga 2, 76)",NaN,-35664780.61,-0.070000,-4.479013e+07,5.094969e+08,6.398590e+08,8.951168,8.941845,0.455290,NaN,307726442.0,4.088293e+08
3,cayuga ct unit 4,"Duke Energy Indiana, LLC",2018.0,None,1001_gt_2018_plant_prime_mover_total_15470,NaN,1001_gt_2018_plant_prime_mover_total_15470,1001_gt_2018_plant_prime_mover_total_15470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,total,1.0,False,4,144,Other,2018-01-01,15470,GT,NG,<NA>,plant_prime_mover,1001,plant_prime_mover,7.0,existing,Natural Gas Fired Combustion Turbine,92,Cayuga,Cayuga GT,True,operating,39022797.0,3.902280e+07,False,PUC,1910495.0,1.910495e+06,0.040093,2018_1001_cayuga ct unit 4_other_92_PUC,"(cayuga 4, 67)",NaN,-2636231.17,-0.050000,-2.636231e+06,5.272462e+07,5.272462e+07,8.551738,8.566081,0.708616,NaN,16338052.0,1.633805e+07
4,cayuga diesel,"Duke Energy Indiana, LLC",2018.0,None,1001_ic_2018_plant_prime_mover_total_15470,NaN,1001_ic_2018_plant_prime_mover_total_15470,1001_ic_2018_plant_prime_mover_total_15470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,total,1.0,False,<NA>,144,Other,2018-01-01,15470,IC,DFO,<NA>,plant_prime_mover,1001,plant_prime_mover,7.0,existing,Petroleum Liquids,92,Cayuga,Cayuga IC,True,operating,1802337.0,1.802337e+06,False,PUC,134223.0,1.342230e+05,0.063422

### Test

In [ ]:
df_test = pd.DataFrame(
    {
        'record_id_test': [
            'record_1', 'record_2', 'record_3',
        ],
        'plant_id_eia': [
            1, 2, 3,
        ],
        'report_date': [
            '2020-01-01', '2020-01-01', '2020-01-01',
        ],
        'plant_part': [
            'plant', 'plant_prime_mover', 'plant_gen',
        ],
        'operational_status_pudl': [
            'operating', 'operating', 'operating',
        ],
        'ferc_acct_name': [
            pd.NA, pd.NA, pd.NA,
        ],
        'unit_id_pudl': [
            pd.NA, pd.NA, pd.NA,
        ],
        'technology_description': [
            pd.NA, pd.NA, 'Conventional Steam Coal',
        ],
        'generator_id': [
            pd.NA, pd.NA, "a",
        ],
        'prime_mover_code': [
            pd.NA, 'ST', 'ST',
        ],
        'energy_source_code_1': [
            pd.NA, pd.NA, pd.NA,
        ],
        'data_col': [
            300, 100, 10,
        ],
        'utility_id_eia': [
            1, 1, 1,
        ],
        'ownership': [
            'total', 'total', 'total',
        ],
        'record_id_eia': [
            'record_id_1_1a_1b_1c', 'reocrd_id_2_2a_2b', 'record_id_3_a',
        ],
    }
).pipe(pudl.helpers.convert_cols_dtypes, 'eia').assign(report_year=lambda x: x.report_date.dt.year)

ppl_test = pd.DataFrame(
    {
        'plant_id_eia': [
            1, 1, 1,
            2, 2,
            3,
        ],
        'report_date': [
            '2020-01-01', '2020-01-01', '2020-01-01',
            '2020-01-01', '2020-01-01',
            '2020-01-01',
        ],
        'plant_part': [
            'plant_gen', 'plant_gen', 'plant_gen',
            'plant_gen', 'plant_gen',
            'plant_gen',
        ],
        'operational_status_pudl': [
            'operating', 'operating', 'operating',
            'operating', 'operating',
            'operating',
        ],
        'ferc_acct_name': [
            'steam', 'steam', 'steam',
            'steam', 'steam',
            'steam',
        ],
        'unit_id_pudl': [
            pd.NA, pd.NA, pd.NA,
            pd.NA, pd.NA,
            pd.NA,
        ],
        'technology_description': [
            'Conventional Steam Coal', 'Conventional Steam Coal',
            'Conventional Steam Coal',
            'Conventional Steam Coal', 'Conventional Steam Coal',
            'Conventional Steam Coal',
        ],
        'generator_id': [
            "1a", "1b", "1c",
            "2a", "2b",
            "a",
        ],
        'prime_mover_code': [
            'ST', 'ST', 'ST',
            'ST', 'ST',
            'ST',
        ],
        'energy_source_code_1': [
            'BIT', 'BIT', 'BIT',
            'BIT', 'BIT',
            'BIT',
        ],
        'utility_id_eia': [
            1, 1, 1,
            1, 1,
            1,
        ],
        'ownership': [
            'total', 'total', 'total',
            'total', 'total',
            'total',
        ],
        'record_id_eia': [
            'record_id_1_1a', 'record_id_1_1b', 'record_id_1_1c',
            'record_id_2_2a', 'record_id_2_2b',
            'record_id_3_a',
        ],
        'capacity_mw': [
            50, 30, 20,
            10, 10,
            100,
        ],
        'net_generation_mwh': [
            10000, 1000, 100,
            250, 200,
            5000,
        ],
        'total_fuel_cost': [
            500, 100, 50,
            250, 200,
            300,
        ],
    }
).pipe(pudl.helpers.convert_cols_dtypes, 'eia').assign(report_year=lambda x: x.report_date.dt.year)

In [ ]:
scale_to_plant_part(
    scale_part='plant_gen',
    df_to_scale=df_test,
    ppl=ppl_test,
    data_set_idx_cols=['record_id_test'],
    data_cols=['data_col'],
    other_cols_to_keep=[],
).reset_index().to_dict()

In [ ]:
scale_part='plant_gen'
df_to_scale=df_test.pipe(pudl.helpers.convert_cols_dtypes, 'eia')
ppl=ppl_test.pipe(pudl.helpers.convert_cols_dtypes, 'eia')
data_set_idx_cols=['record_id_test']
data_cols=['data_col']
other_cols_to_keep=[]

In [ ]:
merged_df = many_merge_on_scale_part(
    plant_part=scale_part,
    df_to_scale=df_to_scale,
    cols_to_keep=data_cols + data_set_idx_cols + other_cols_to_keep,
    ppl=ppl.reset_index()
)

# grab all of the ppl columns, plus data set's id column(s)
# this enables us to have a unique index
idx_cols = (
    pudl.analysis.plant_parts_eia.PLANT_PARTS[scale_part]['id_cols']
    + pudl.analysis.plant_parts_eia.IDX_TO_ADD
    + pudl.analysis.plant_parts_eia.IDX_OWN_TO_ADD
    + data_set_idx_cols
)
scaled_df = merged_df.set_index(idx_cols)

### The Merge

In [582]:
scaled_fe = scale_to_plant_part(
    scale_part = 'plant_gen',
    df_to_scale = fe,
    ppl=ppl,
    data_set_idx_cols = ['record_id_ferc1'],
    data_cols = [
        'capex_total',
        'capex_annual_addt',
        'opex_nonfuel',
        'capacity_mw_ferc1'
    ],
    other_cols_to_keep=['avg_num_employees']
)

scaled_de = scale_to_plant_part(
    scale_part = 'plant_gen',
    df_to_scale = pudl_rmi.make_plant_parts_eia.reassign_id_ownership_dupes(de),
    ppl=ppl,
    data_set_idx_cols = ['line_id'],
    data_cols = [
        'plant_balance_w_common',
        'book_reserve_w_common',
        'unaccrued_balance_w_common',
        'net_salvage_w_common',
        'depreciation_annual_epxns_w_common'
    ],
    other_cols_to_keep=[
        'utility_name_ferc1',
        'net_removal_rate',
        'depreciation_annual_rate',
        'remaining_life_avg',
        'data_source',
    ]
)

Pre-scale aggregate.
Post-scale aggregate.
Summing: ['capex_total_scaled', 'capex_annual_addt_scaled', 'opex_nonfuel_scaled', 'capacity_mw_ferc1_scaled'].
Averaging: {'avg_num_employees': 'capacity_mw_ferc1_scaled'}.
Pre-scale aggregate.
Post-scale aggregate.
Summing: ['plant_balance_w_common_scaled', 'book_reserve_w_common_scaled', 'unaccrued_balance_w_common_scaled', 'net_salvage_w_common_scaled', 'depreciation_annual_epxns_w_common_scaled'].
Averaging: {'net_removal_rate': 'unaccrued_balance_w_common_scaled', 'depreciation_annual_rate': 'unaccrued_balance_w_common_scaled', 'remaining_life_avg': 'unaccrued_balance_w_common_scaled'}.


In [585]:
df = scaled_de.reset_index()
df[
    (df.report_date.dt.year == 2020)
    & (df.utility_id_pudl.isin([90,97]))
]

,plant_id_eia,generator_id,report_date,operational_status_pudl,utility_id_eia,ownership,record_id_eia,line_id,data_source,plant_balance_w_common_scaled,book_reserve_w_common_scaled,unaccrued_balance_w_common_scaled,net_salvage_w_common_scaled,depreciation_annual_epxns_w_common_scaled,net_removal_rate,depreciation_annual_rate,remaining_life_avg,utility_name_ferc1,plant_part,unit_id_pudl,prime_mover_code,energy_source_code_1,technology_description,ferc_acct_name,true_gran,appro_part_label,appro_record_id_eia,capacity_eoy_mw,capacity_factor,capacity_mw,fraction_owned,fuel_cost_per_mmbtu,fuel_cost_per_mwh,fuel_type_code_pudl,heat_rate_mmbtu_mwh,installation_year,net_generation_mwh,operational_status,ownership_dupe,planned_retirement_date,plant_id_pudl,plant_name_eia,plant_name_new,plant_part_id_eia,record_count,retirement_date,total_fuel_cost,total_mmbtu,utility_id_pudl,report_year,plant_id_report_year
592,2706,1,2020-01-01,operating,3046,total,2706_1_2020_plant_gen_total_3046,2020_2706_asheville unit 1_steam_97_FERC,FERC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Duke Energy Progress, LLC",plant_gen,1,ST,BIT,Conventional Steam Coal,Steam,False,plant_unit,2706_1_2020_plant_unit_total_3046,0.0,NaN,206.6,1.0,NaN,NaN,coal,NaN,<NA>,NaN,retired,False,NaT,25,Asheville,Asheville 1,2706_1_plant_gen_total_3046,8.0,2020-01-01,NaN,NaN,97,2020,25_2020
596,2706,2,2020-01-01,operating,3046,total,2706_2_2020_plant_gen_total_3046,2020_2706_asheville unit 2_steam_97_FERC,FERC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Duke Energy Progress, LLC",plant_gen,2,ST,BIT,Conventional Steam Coal,Steam,False,plant_unit,2706_2_2020_plant_unit_total_3046,0.0,NaN,207.0,1.0,NaN,NaN,coal,NaN,<NA>,NaN,retired,False,NaT,25,Asheville,Asheville 2,2706_2_plant_gen_total_3046,8.0,2020-01-01,NaN,NaN,97,2020,25_2020
600,2706,GT1,2020-01-01,operating,3046,total,2706_gt1_2020_plant_gen_total_3046,2020_2706_asheville ic turbin_other_97_FERC,FERC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Duke Energy Progress, LLC",plant_gen,<NA>,GT,NG,Natural Gas Fired Combustion Turbine,Other,True,plant_gen,2706_gt1_2020_plant_gen_total_3046,211.7,0.047894,211.7,1.0,NaN,NaN,gas,NaN,<NA>,8.906296e+04,existing,False,NaT,25,Asheville,Asheville GT1,2706_GT1_plant_gen_total_3046,8.0,NaT,NaN,NaN,97,2020,25_2020
604,2706,GT2,2020-01-01,operating,3046,total,2706_gt2_2020_plant_gen_total_3046,2020_2706_asheville ic turbin_other_97_FERC,FERC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Duke Energy Progress, LLC",plant_gen,<NA>,GT,NG,Natural Gas Fired Combustion Turbine,Other,True,plant_gen,2706_gt2_2020_plant_gen_total_3046,211.8,0.047894,211.8,1.0,NaN,NaN,gas,NaN,<NA>,8.910504e+04,existing,False,NaT,25,Asheville,Asheville GT2,2706_GT2_plant_gen_total_3046,8.0,NaT,NaN,NaN,97,2020,25_2020
608,2707,1,2020-01-01,operating,3046,total,2707_1_2020_plant_gen_total_3046,2020_2707_blewett_hydraulic_97_FERC,FERC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Duke Energy Progress, LLC",plant_gen,<NA>,HY,WAT,Conventional Hydroelectric,Hydraulic,True,plant_gen,2707_1_2020_plant_gen_total_3046,3.2,0.134835,3.2,1.0,NaN,NaN,hydro,NaN,<NA>,3.790049e+03,existing,False,NaT,60,Blewett,Blewett 1,2707_1_plant_gen_total_3046,10.0,NaT,NaN,NaN,97,2020,60_2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2491,7981,7,2020-01-01,operating,5416,total,7981_7_2020_plant_gen_total_5416,2020_7981_mill creek_other_90_PUC,PUC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Duke Energy Carolinas, LLC",plant_gen,<NA>,GT,NG,Natural Gas Fired Combustion Turbine,Other,True,plant_gen,7981_7_2020_plant_gen_total_5416,99.9,0.010019,99.9,1.0,NaN,NaN,gas,NaN,<NA>,8.791500e+03,existing,False,NaT,377,Mill Creek (SC),Mill Creek (SC) 7,7981_7_plant_gen_total_5416,8.0,NaT,NaN,NaN,90,2020,377_2020
2496,7981,8,2020-01-01,operating,5416,total,7981_8_2020_plant_gen_total_5416,2020_7981_mill creek ct_other_90_FERC,FERC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Duke Energ

In [ ]:
df.utility_id_pudl

### Duke ACR

In [524]:
coal_pounds_dep = {
    2706: 
        98_220_932, # asheville
    # 0: 
    #     33_631_199, # Cape Fear plant - already retired;
    58215: 
        9_207_711, # h.f. lee/wayne
    58697: 
        186_376_226, # sutton
    2716: 
        6_044_240, # weatherspoon
}

# DEC id_ferc1 = 45 / id_pudl = 90
# 2019 - 2018
dec_2018 = 886_954*1000
dec_2019 = 2_718_147*1000
diff_dec = dec_2019 - dec_2018

# DEP id_fer1 = 17 / id_pudl = 97
# 2019 - 2018
dep_2018 = 827_197.089*1000
dep_2019 = 1_622_833.321*1000
diff_dep = dep_2019 - dep_2018

# dec Riverbend, Buck, and Dan River

In [525]:
arc = (
    pd.DataFrame(
        coal_pounds_dep.values(), 
        index=coal_pounds_dep.keys(),
        columns=['arc_proxy']
    )
    .reset_index()
    .rename(columns={'index': 'plant_id_eia'})
    .assign(
        arc_rate=lambda x: x.arc_proxy / x.arc_proxy.sum(),
        arc_by_plant=lambda x: x.arc_rate* diff_dep
    )
)
assert(arc.arc_rate.sum() == 1)
assert(arc.arc_by_plant.sum() == diff_dep)

In [527]:
arc_e = pd.merge(
    ppl[
        (ppl.plant_part == 'plant')
        & (ppl.report_year == 2019)
        & (ppl.plant_id_eia.isin(coal_pounds_dep.keys()))
        & (ppl.ownership == 'total')
        & (ppl.operational_status_pudl =='operating')
    ].reset_index(),
    arc,
    on=['plant_id_eia']
)

scaled_arc = scale_to_plant_part(
    scale_part = 'plant_gen',
    df_to_scale = arc_e.reset_index(),
    ppl=ppl,
    data_set_idx_cols = ['index'],
    data_cols = [
        'arc_by_plant',
        'arc_proxy',
    ],
    other_cols_to_keep=[],
    skip_agg=True
)
assert(np.isclose(scaled_arc.arc_by_plant_scaled.sum(), diff_dep))

### Merging scaled dfs for outputs

In [599]:
output = (
    pd.merge(
        scaled_de,
        scaled_fe,
        right_index=True, left_index=True,
        how='outer',
        suffixes=('_deprish', '_ferc1'),
        indicator=True
    ) 
    .merge(
        scaled_arc[['arc_by_plant_scaled']],
        right_index=True, left_index=True,
        how='left',
    )
    # is there a better way to do this? record_id_eia is already in the previous index
    .reset_index().set_index(['record_id_eia'])
)
# choose data for the following columns from either the depreciation data
# or the ferc1 data.
cols_to_choose = [
    'plant_name_eia',
    'fraction_owned',
    'technology_description',
    'operational_status',
    'utility_id_pudl',
    'plant_id_pudl'
]
output = output.drop(
    columns=[f"{x}_deprish" for x in cols_to_choose] + [f"{x}_ferc1" for x in cols_to_choose])
# add some ppl columns back into the output
ppl_cols_to_add = cols_to_choose + [
    'total_fuel_cost', 'installation_year', 'net_generation_mwh',
    'capacity_factor', 'capacity_mw', 
]
output = output.merge(
    ppl[ppl_cols_to_add],
    right_index=True, left_index=True,
    how='left'
)

### Duke mode output

In [591]:
cols = [
    'utility_name_ferc1',
    'report_date',
    'data_source',
    'utility_id_pudl',
    'plant_id_pudl',
    'plant_name_eia',
    'generator_id',
    'plant_balance_w_common_scaled',
    'book_reserve_w_common_scaled',
    'unaccrued_balance_w_common_scaled',
    'depreciation_annual_epxns_w_common_scaled',
    'depreciation_annual_rate',
    'net_salvage_w_common_scaled',
    'record_id_ferc1',
    'record_id_eia',
    'capex_total_scaled',
    'capex_annual_addt_scaled',
    'arc_by_plant_scaled'
]

duke_out = (
    output[
        output.utility_id_pudl.isin([90,97])
        & (output.report_date.dt.year.isin([2018, 2019, 2020]))
        & (output.data_source == 'FERC')
    ]
    .reset_index()
    .pipe(pudl.helpers.organize_cols, cols)
    .sort_values(['utility_name_ferc1','plant_name_eia', 'generator_id', 'report_date', ])
)

In [594]:
duke_out.to_csv(pudl_rmi.OUTPUTS_DIR/ 'duke_for_modernization.csv', index=False)

#### ARC Exploration

In [406]:
d_dep = d[
    (d.utility_id_pudl == 97)
    & (d.report_date.dt.year == 2018)
    & (d.data_source =='FERC')
]

d_dec = d[
    (d.utility_id_pudl == 90)
    & (d.report_date.dt.year == 2018)
    & (d.data_source =='FERC')
]

ns_dep = d_dep.net_salvage.sum()
ns_dec = d_dec.net_salvage.sum()

ns_plants_dep = - d_dep[
    d_dep.plant_id_eia.notnull()
].net_salvage.sum()
ns_plants_dec = - d_dec[d_dec.plant_id_eia.notnull()].net_salvage.sum()

logger.info(
    f"DEC net salvage/ARC 2018:            {ns_plants_dec/dec_2018:.0%}\n"
    f"DEP net salvage/ARC 2018:            {ns_plants_dep/dep_2018:.0%}\n"
    f"SC PUC Plant-level DEP ARC/ARC 2018: {sum(coal_pounds_dep.values()) / (dep_2018):.0%}"
)

DEC net salvage/ARC 2018:            101%
DEP net salvage/ARC 2018:            95%
SC PUC Plant-level DEP ARC/ARC 2018: 36%


### Export Duke Jawn

In [601]:
duke_out = pudl_rmi.model_inputs.convert_to_model_format(
    output.reset_index().assign(report_year=lambda x:  x.report_date.dt.year),
    pudl_out,
    util_ids_pudl=[90,97],
    years=[2018,2019,2020]
)
duke_out = duke_out.replace(
    {'Utility': {
        'Duke Energy Progress - (NC)': 'Duke Energy Progress',
        'Duke Energy Carolinas, LLC': 'Duke Energy Carolinas'},
    }
)
duke_out['Asset Status'] = duke_out['Asset Status'].str.capitalize()


duke_out.to_csv(pudl_rmi.OUTPUTS_DIR / f"current_owned_duke_{date.today().strftime('%Y-%m-%d')}.csv", index=False)

## Make the output directly

In [ ]:
inputs = InputsManager(
    plant_parts_eia=rmi_out.grab_plant_part_list(),
    deprish_eia=rmi_out.grab_deprish_to_eia(clobber=False),
    ferc1_to_eia=rmi_out.grab_ferc1_to_eia(clobber=False),
)
matcher = MatchMaker(inputs)
scaler = Scaler(matcher)